In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Github/

/content/drive/MyDrive/Github


In [ ]:
# !rm -rf .git
# !git clone https://github.com/bst-depractice/spark_play

Cloning into 'spark_play'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (15/15), 1.17 MiB | 4.35 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [4]:
!apt-get -qq update > /tmp/apt.out
!apt-get install -y -qq openjdk-11-jdk-headless

In [5]:
!(wget -q --show-progress -nc https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz)
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [6]:
try:
  import pyspark, findspark, delta, pyngrok
except:
  %pip install -q --upgrade pyspark==3.2.1
  %pip install -q findspark
  %pip install -q delta
  %pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 kB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Pass the config k,v pairs and get a spark session object

In [7]:
import findspark
import pyspark
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/drive/MyDrive/Github/spark-3.2.1-bin-hadoop3.2"

findspark.init()
MAX_MEMORY="8g"
maven_coords = [
    "org.apache.spark:spark-avro_2.12:3.2.1",
    "io.delta:delta-core_2.12:2.0.0rc1",
    "org.xerial:sqlite-jdbc:3.36.0.3",
    "graphframes:graphframes:0.8.2-spark3.2-s_2.12",
    "com.acervera.osm4scala:osm4scala-spark3-shaded_2.12:1.0.8",
]
spark = (pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.jars.packages", ",".join(maven_coords))
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.executor.memory", MAX_MEMORY)
    .config("spark.driver.memory", MAX_MEMORY)
    .config('spark.ui.port', '4050')
    .enableHiveSupport()
    .getOrCreate()
    )
spark

In [ ]:
from pyngrok import ngrok, conf
import getpass

print("Enter your authtoken, which can be copied "
"from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = getpass.getpass()

ui_port = 4040
public_url = ngrok.connect(ui_port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{ui_port}\"")

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
··········


 * ngrok tunnel "https://50f0-34-68-94-133.ngrok-free.app" -> "http://127.0.0.1:4040"


Setup is complete. At this point you have started a spark application and able to access the application-ui using the url above.
You can start writing your data transformation code below...

### Add Schema to your Dataframe

StructType, StructField, Spark Datatypes

Spark Datatypes are not the same as Python Datatypes

string vs StringType<br>
int vs IntegerType<br>
double vs DoubleType<br>
date vs DateType<br>

The type classes are in pyspark.sql.types<br>
`from pyspark.sql.types import *`

#### How to create an empty Dataframe for a given schema ?


In [ ]:
from pyspark.sql.types import *

from datetime import datetime
schema = StructType(
          [StructField("name", StringType(), False),
           StructField("dob", DateType(), False)
          ]
        )

df = spark.createDataFrame([["ash", datetime.strptime("2020-01-01", "%Y-%m-%d")]], schema = schema)
df.show()
df.printSchema()

# Pyspark dataframe API: Create a dataframe from the csv data source

Use pyspark syntax to do data transformation

In [8]:
movies_df = spark.read.format("csv").option("header", "true").load('./spark_play/netflix_titles.csv')
movies_df.show(5)

+-------+-------+-----+-----------------+--------------------+-------------+-----------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|title|         director|                cast|      country|       date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+-----+-----------------+--------------------+-------------+-----------------+------------+------+---------+--------------------+--------------------+
|     s1|TV Show|   3%|             null|João Miguel, Bian...|       Brazil|  August 14, 2020|        2020| TV-MA|4 Seasons|International TV ...|In a future where...|
|     s2|  Movie| 7:19|Jorge Michel Grau|Demián Bichir, Hé...|       Mexico|December 23, 2016|        2016| TV-MA|   93 min|Dramas, Internati...|After a devastati...|
|     s3|  Movie|23:59|     Gilbert Chan|Tedd Chan, Stella...|    Singapore|December 20, 2018|        2011|     R|   78 min|Horror Movies, In...|When an army recr...

#### Dataframe Column Operations
Add a new Column from existing columns<br>
Modify a column<br>
Rename a column <br>

#### How do you refer a column ?<br>
use df.column <br>
use col object

from pyspark.sql.functions import col

use when condition in pyspark api for complex conditions

#### Try these other functions
concat, upper, lower




In [21]:
movies_df.withColumn("isDrama", movies_df.listed_in.contains('Drama'))
movies_df.withColumn("type", movies_df.title)
movies_df.withColumnRenamed("country", "released_country")

from pyspark.sql.functions import col, lit, when

movies_df.withColumn("pre-2000", col("release_year"))

movies_df.withColumn("pre-2000", when(col("release_year")<2010, "pre-2k").otherwise("post_2000")  )

movies_df.withColumn("source", lit("csv"))





DataFrame[show_id: string, type: string, title: string, director: string, cast: string, country: string, date_added: string, release_year: string, rating: string, duration: string, listed_in: string, description: string, source: string]

####Add a prefix to all column names


In [ ]:
toDF(["movie_"+x for x in movies_df.columns])

#### collect df values as python lists ❌::

In [62]:
emp = spark.createDataFrame([["ash", "2019-01-01", "HR"], ["mod", "2020-01-01", "IT"]], schema=("name", "dob", "dept"))
emp.show()

+----+----------+----+
|name|       dob|dept|
+----+----------+----+
| ash|2019-01-01|  HR|
| mod|2020-01-01|  IT|
+----+----------+----+



In [42]:
a= emp.select('name').collect()
print(type(a))
print(type(a[0]))
print(a[0])
print(a[0][0])
print(a[1][0])
print(a)
[_[0] for _ in a]

<class 'list'>
<class 'pyspark.sql.types.Row'>
Row(name='ash')
ash
mod
[Row(name='ash'), Row(name='mod')]


['ash', 'mod']

#### Defining UDFs

import udf function and return types


use lambda function <br>
use python function <br>

register to use in spark.sql

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
first_word_udf = udf(lambda x: x.split()[0], StringType())

def get_first_word(x):
  return x.split()[0]

first_word_udf = udf(get_first_word, StringType())
movies_df.withColumn("first", first_word_udf(col("listed_in"))) .select("listed_in" ,'first')

movies_df.createOrReplaceGlobalTempView("sql_movie")
spark.udf.register("super_udf", get_first_word, StringType())
spark.sql("select  listed_in, super_udf(listed_in) from sql_movie").show()

#### JOIN
pyspark syntax

df1.join(df2, on=[col1], how="inner")

```
Above works, when both dataframes have the same join columns (name matching).
Note the repeating columns in the result.
What about the join condition column(s)?
```

when you want to join on columns with different names.


Selcet all columns from one side in simpler way




In [1]:
# drama_df = movies_df.filter(col("listed_in").contains('Drama'))
# inner_df =  movies_df.join(drama_df, on=["release_year"])


# drama_df = drama_df.withColumnRenamed("release_year", "year_produced")
# inner_df =  movies_df.join(drama_df, on=[movies_df.release_year==drama_df.year_produced]).select(drama_df.*).show(5)

drama_df = drama_df.withColumnRenamed("release_year", "year_produced")
movies_df.join(drama_df, on=[movies_df.release_year==drama_df.year_produced]) .select(drama_df["*"], movies_df.release_year ).show(5)


NameError: name 'drama_df' is not defined

#### Try other join types.
1. left, anti <br>

2. Check exclude

## Run the below exercises with pyspark API. You may verify your results with SQL API.


#### 1. Try filtering by multiple conditions- say 1)year 2010 OR country = India 2) year 2010 and country India

#### 2. Get a peek of dataframe by show, first, last

#### 3. Select 1 or more columns and drop the duplicates. (there is a simple function)

#### 4. Check the groupby syntax. First groupby(col) add aggregations to it as agg(avg(col))

#### 5. Can you get the distinct count of countries in each release year . Produce a dataframe showing year, count_countries

#### 6. What is the semi join vs anti join types ? Show an example for each. If needed create a simple dataframe using createDataframe method.

#### 7. Create a single row dataframe using ['20200714', '2020-07-15', '2020-16-07 23:15']. Convert each of the columns to the date/timestamp values formats applicable. After conversion, printing the target dataframe's schema will be as follows.

```
source
 |-- a: string (nullable = true)
 |-- b: string (nullable = true)
 |-- c: string (nullable = true)

target
 |-- a: date (nullable = true)
 |-- b: date (nullable = true)
 |-- c: timestamp (nullable = true)
```






### JSON in .json

dot notation

explode

In [97]:
from pyspark.sql.functions import explode

row = spark.read.format("json").load("./spark_play/data/jsonfile.json")

json_image_schema = row.schema

row.show(truncate=False)

row.select( row["Image"]["Title"].alias("img_title"), row["Image"]["Height"], row["Image"]["Width"]).show()

row.select( explode(row["Image"]["IDs"]).alias("ID"), row["Image"]["Height"] ).show()


+------------------------------------------------------------------------------------------------------------+
|Image                                                                                                       |
+------------------------------------------------------------------------------------------------------------+
|{600, [116, 943, 234, 38793], {125, http://www.example.com/image/481989943, 100}, View from 15th Floor, 800}|
+------------------------------------------------------------------------------------------------------------+

+--------------------+------------+-----------+
|           img_title|Image.Height|Image.Width|
+--------------------+------------+-----------+
|View from 15th Floor|         600|        800|
+--------------------+------------+-----------+

+-----+------------+
|   ID|Image.Height|
+-----+------------+
|  116|         600|
|  943|         600|
|  234|         600|
|38793|         600|
+-----+------------+



### JSON in txt

In [1]:
from pyspark.sql.functions import explode, from_json

arow = (spark.read.format("csv")
      .option("delimiter", '|')
      .option("header", "true")
      .option("quote", '\'')
      .load("./spark_play/data/json_in_text.csv")
)

arow.printSchema()

get_json_schema = spark.read.json(arow.json_image).schema

arow.withColumn("json_image",  from_json(arow["image"], schema=get_json_schema )).select(col("image_id"), col("json_image.Image.Title")).show()


ModuleNotFoundError: No module named 'pyspark'